In [3]:
import pandas as pd

# Load the CSV file.
df = pd.read_csv("M2.csv")

# Print the columns for inspection.
print("Columns in the CSV:")
print(df.columns.tolist())

# Optionally, print a few rows to check the data.
print("\nSample data:")
print(df.head())


Columns in the CSV:
['Date', 'Batting Team', 'Bowling Team', 'Striker', 'Non-Striker', 'Bowler', 'Innings', 'Unique', 'Over Number', 'Over Ball', 'Runs', 'Wides', 'Team Runs', 'Remaining Overs', 'Lead By', 'Trail By', 'Fall Of Wickets', 'Day', 'Session', 'Leg Byes', 'No Balls', 'Balls', 'Byes', 'Remaining Balls', 'Remaining Runs', 'Run Rate', 'Batter Sixes', 'Batter Runs', 'Batter Fours', 'Batter Balls', 'Batter Total Runs', 'Dismissal Type', 'Dismissal Fielder', 'On Field Minutes', 'Shot Angle', 'Shot Magnitude', 'Wagon Position', 'Pitching Length', 'Pitching Line', 'Shot Connection', 'False Shot', 'Feet Movement', 'Shot Type', 'Delivery Type', 'Bowling Side', 'Bowling Type', 'Bowling Hand', 'Catch Difficulty', 'Bounce_Angle', 'Pitch_x', 'Pitch_y', 'Friction', 'Restitution', 'Deviation', 'Reaction Time', 'Release Speed', 'Swing', 'Swing Distance', 'Six Distance']

Sample data:
                 Date         Batting Team      Bowling Team          Striker  \
0  2025-3-24 15:30:00  Sunri

In [5]:
import numpy as np

def strike_rate_bonus(runs, balls):
    """
    Calculate strike rate bonus.
    Strike Rate (SR) = (runs * 100) / balls.
    Returns a tuple: (bonus, sr)
    """
    if balls == 0:
        return 0, np.nan
    sr = (runs * 100) / balls
    if sr > 170:
        return 6, sr
    elif 150 <= sr <= 170:
        return 4, sr
    elif 130 <= sr < 150:
        return 2, sr
    elif 60 <= sr < 70:
        return -2, sr
    elif 50 <= sr < 60:
        return -4, sr
    elif sr < 50:
        return -6, sr
    return 0, sr

#sorting
df_sorted = df.sort_values(by=["Innings", "Striker", "Over Number", "Over Ball"])

# For batting, we assume the “Batter Balls” column is updated on each legal delivery.
# Therefore, to get the final score and balls faced for each batsman per innings,
# we group by [Innings, Striker] and take the last entry.
final_batting = df_sorted.groupby(["Innings", "Striker"]).agg(
    final_score=pd.NamedAgg(column="Batter Total Runs", aggfunc="last"),
    balls_faced=pd.NamedAgg(column="Batter Balls", aggfunc="last")
).reset_index()

# Now, for boundaries and sixes we use the data from each ball.
# (Assuming each ball row indicates if a boundary was hit.)
# We assume that in each ball, the columns "Batter Fours" and "Batter Sixes" indicate whether that ball produced a four or six.
# If they are binary (0/1) on each ball, summing them per innings gives the total.
boundaries = df_sorted.groupby(["Innings", "Striker"])["Batter Fours"].sum().reset_index(name="boundaries")
sixes = df_sorted.groupby(["Innings", "Striker"])["Batter Sixes"].sum().reset_index(name="sixes")

# Merge the batting aggregates.
batting_stats = pd.merge(final_batting, boundaries, on=["Innings", "Striker"], how="left")
batting_stats = pd.merge(batting_stats, sixes, on=["Innings", "Striker"], how="left")
batting_stats[["boundaries", "sixes"]] = batting_stats[["boundaries", "sixes"]].fillna(0)

# Apply bonus rules:
batting_stats["bonus_30"] = np.where(batting_stats["final_score"] >= 30, 4, 0)
batting_stats["bonus_half_century"] = np.where(
    (batting_stats["final_score"] >= 50) & (batting_stats["final_score"] < 100), 8, 0
)
batting_stats["bonus_century"] = np.where(batting_stats["final_score"] >= 100, 16, 0)

# To check for duck penalty, we need the dismissal info.
# We extract the last ball for each [Innings, Striker] (which should contain the final dismissal type, if any)
last_ball = df_sorted.groupby(["Innings", "Striker"]).last().reset_index()
batting_stats = pd.merge(batting_stats, last_ball[["Innings", "Striker", "Dismissal Type"]], on=["Innings", "Striker"], how="left")
batting_stats["duck_penalty"] = np.where((batting_stats["final_score"] == 0) & (batting_stats["Dismissal Type"].notnull()), -2, 0)

# Calculate strike rate bonus.
strike_results = batting_stats.apply(lambda row: strike_rate_bonus(row["final_score"], row["balls_faced"]), axis=1)
batting_stats["bonus_strike_rate"], batting_stats["strike_rate"] = zip(*strike_results)

# Bonus for boundaries and sixes:
batting_stats["bonus_boundaries"] = batting_stats["boundaries"] * 1
batting_stats["bonus_sixes"] = batting_stats["sixes"] * 2

# Total batting points (sum of runs and all bonuses):
batting_stats["batting_points"] = (
    batting_stats["final_score"]
    + batting_stats["bonus_boundaries"]
    + batting_stats["bonus_sixes"]
    + batting_stats["bonus_30"]
    + batting_stats["bonus_half_century"]
    + batting_stats["bonus_century"]
    + batting_stats["duck_penalty"]
    + batting_stats["bonus_strike_rate"]
)

print("=== Batting Points Breakdown ===")
print(batting_stats[["Innings", "Striker", "final_score", "balls_faced", "boundaries",
                      "sixes", "bonus_30", "bonus_half_century", "bonus_century", "duck_penalty",
                      "strike_rate", "bonus_strike_rate", "batting_points"]])


=== Batting Points Breakdown ===
    Innings             Striker  final_score  balls_faced  boundaries  sixes  \
0         1     Abhinav Manohar            0            1           0      0   
1         1     Abhishek Sharma           24           11          32      0   
2         1        Aniket Verma            7            3           0      2   
3         1    Heinrich Klaasen           34           14          35      8   
4         1        Ishan Kishan          106           47         332    102   
5         1  Nitish Kumar Reddy           30           15          43      6   
6         1         Pat Cummins            0            1           0      0   
7         1         Travis Head           67           31         143     75   
8         2         Dhruv Jurel           70           35         146    106   
9         2        Jofra Archer            1            1           0      0   
10        2         Nitish Rana           11            8          10      0   
11     

In [6]:
import pandas as pd
import numpy as np

# Helper function for calculating strike rate bonus.
def strike_rate_bonus(runs, balls):
    """
    Calculate strike rate bonus.
    Strike Rate (SR) = (runs * 100) / balls.
    Returns a tuple: (bonus, sr).
    """
    if balls == 0:
        return 0, np.nan
    sr = (runs * 100) / balls
    if sr > 170:
        return 6, sr
    elif 150 <= sr <= 170:
        return 4, sr
    elif 130 <= sr < 150:
        return 2, sr
    elif 60 <= sr < 70:
        return -2, sr
    elif 50 <= sr < 60:
        return -4, sr
    elif sr < 50:
        return -6, sr
    return 0, sr

# Sort the dataframe by Innings, Striker, Over Number, and Over Ball.
df_sorted = df.sort_values(by=["Innings", "Striker", "Over Number", "Over Ball"])

# For batting, we assume the cumulative values are updated on each legal delivery.
# We group by [Innings, Striker] and take the final record (last legal delivery) as the final state.
# We use the following columns from the final record:
#   - "Batter Total Runs" → final_score
#   - "Batter Balls"       → balls_faced
#   - "Batter Fours"       → cumulative fours
#   - "Batter Sixes"       → cumulative sixes
final_batting = df_sorted.groupby(["Innings", "Striker"]).agg(
    final_score = pd.NamedAgg(column="Batter Total Runs", aggfunc="last"),
    balls_faced = pd.NamedAgg(column="Batter Balls", aggfunc="last"),
    boundaries_total = pd.NamedAgg(column="Batter Fours", aggfunc="last"),
    sixes_total = pd.NamedAgg(column="Batter Sixes", aggfunc="last")
).reset_index()

# Apply bonus rules based on the final score.
final_batting["bonus_30"] = np.where(final_batting["final_score"] >= 30, 4, 0)
final_batting["bonus_half_century"] = np.where(
    (final_batting["final_score"] >= 50) & (final_batting["final_score"] < 100), 8, 0
)
final_batting["bonus_century"] = np.where(final_batting["final_score"] >= 100, 16, 0)

# To determine duck penalty we need to check if the batsman was dismissed for 0.
# We take the last legal ball (already used above) to extract dismissal info.
last_ball = df_sorted.groupby(["Innings", "Striker"]).last().reset_index()
final_batting = pd.merge(final_batting, last_ball[["Innings", "Striker", "Dismissal Type"]],
                         on=["Innings", "Striker"], how="left")
final_batting["duck_penalty"] = np.where(
    (final_batting["final_score"] == 0) & (final_batting["Dismissal Type"].notnull()), -2, 0
)

# Calculate strike rate bonus.
strike_results = final_batting.apply(lambda row: strike_rate_bonus(row["final_score"], row["balls_faced"]), axis=1)
final_batting["bonus_strike_rate"], final_batting["strike_rate"] = zip(*strike_results)

# For boundaries and sixes, use the final cumulative values:
# - Each boundary gives +1 point.
# - Each six gives +2 points.
final_batting["bonus_boundaries"] = final_batting["boundaries_total"] * 1
final_batting["bonus_sixes"] = final_batting["sixes_total"] * 2

# Total Batting Points calculation:
final_batting["batting_points"] = (
    final_batting["final_score"]
    + final_batting["bonus_boundaries"]
    + final_batting["bonus_sixes"]
    + final_batting["bonus_30"]
    + final_batting["bonus_half_century"]
    + final_batting["bonus_century"]
    + final_batting["duck_penalty"]
    + final_batting["bonus_strike_rate"]
)

# Display the batting points breakdown.
print("=== Revised Batting Points Breakdown ===")
print(final_batting[["Innings", "Striker", "final_score", "balls_faced", "boundaries_total", "sixes_total",
                     "bonus_30", "bonus_half_century", "bonus_century", "duck_penalty",
                     "strike_rate", "bonus_strike_rate", "batting_points"]])


=== Revised Batting Points Breakdown ===
    Innings             Striker  final_score  balls_faced  boundaries_total  \
0         1     Abhinav Manohar            0            1                 0   
1         1     Abhishek Sharma           24           11                 5   
2         1        Aniket Verma            7            3                 0   
3         1    Heinrich Klaasen           34           14                 5   
4         1        Ishan Kishan          106           47                11   
5         1  Nitish Kumar Reddy           30           15                 4   
6         1         Pat Cummins            0            1                 0   
7         1         Travis Head           67           31                 9   
8         2         Dhruv Jurel           70           35                 5   
9         2        Jofra Archer            1            1                 0   
10        2         Nitish Rana           11            8                 2   
11        2

In [7]:
# Helper function for economy bonus.
def economy_bonus(runs_conceded, balls_bowled):
    """
    Computes the economy bonus for bowlers.
    Economy = runs_conceded / (balls_bowled/6)
    Returns (bonus, economy_rate).
    """
    if balls_bowled == 0:
        return 0, float('nan')
    overs = balls_bowled / 6.0
    econ = runs_conceded / overs
    if econ < 5:
        return 6, econ
    elif 5 <= econ < 6:
        return 4, econ
    elif 6 <= econ < 7:
        return 2, econ
    elif 10 <= econ < 11:
        return -2, econ
    elif 11 <= econ < 12:
        return -4, econ
    elif econ >= 12:
        return -6, econ
    return 0, econ

# Filter legal deliveries for bowling – legal means Wides == 0 and No Balls == 0.
legal_bowling = df_sorted[(df_sorted["Wides"] == 0) & (df_sorted["No Balls"] == 0)]

# Group by Bowler to count legal balls and total runs conceded.
bowling_stats = legal_bowling.groupby("Bowler").agg(
    balls_bowled=pd.NamedAgg(column="Over Ball", aggfunc="count"),
    runs_conceded=pd.NamedAgg(column="Runs", aggfunc="sum")
).reset_index()

# Count wickets: we count a wicket if "Dismissal Type" is not null and not "run out".
wickets_df = legal_bowling[(legal_bowling["Dismissal Type"].notnull()) & (legal_bowling["Dismissal Type"] != "run out")]
wicket_counts = wickets_df.groupby("Bowler").size().reset_index(name="wickets")

# Bonus for LBW or bowled dismissals.
lbw_bowled_df = legal_bowling[legal_bowling["Dismissal Type"].isin(["lbw", "bowled"])]
lbw_bowled_counts = lbw_bowled_df.groupby("Bowler").size().reset_index(name="lbw_bowled")

# Merge these bowling aggregates.
bowling_stats = pd.merge(bowling_stats, wicket_counts, on="Bowler", how="left")
bowling_stats = pd.merge(bowling_stats, lbw_bowled_counts, on="Bowler", how="left")
bowling_stats[["wickets", "lbw_bowled"]] = bowling_stats[["wickets", "lbw_bowled"]].fillna(0)

# Wicket bonus function (only applied once per match)
def wicket_bonus(n):
    if n >= 5:
        return 16
    elif n == 4:
        return 8
    elif n == 3:
        return 4
    else:
        return 0

bowling_stats["bonus_wickets"] = bowling_stats["wickets"].apply(wicket_bonus)

# Compute maiden overs:
# Group legal deliveries by Bowler, Innings, and Over Number.
over_runs = legal_bowling.groupby(["Bowler", "Innings", "Over Number"]).agg(
    runs_in_over=pd.NamedAgg(column="Runs", aggfunc="sum")
).reset_index()

# An over is maiden if runs_in_over is 0.
over_runs["is_maiden"] = (over_runs["runs_in_over"] == 0).astype(int)
maiden_over = over_runs.groupby("Bowler").agg(
    maiden_overs=pd.NamedAgg(column="is_maiden", aggfunc="sum")
).reset_index()

bowling_stats = pd.merge(bowling_stats, maiden_over, on="Bowler", how="left")
bowling_stats["maiden_overs"] = bowling_stats["maiden_overs"].fillna(0)

# Now compute bowling points:
# Each wicket gives +25 points.
bowling_stats["points_wickets"] = bowling_stats["wickets"] * 25
# Each LBW or bowled gives +8 points.
bowling_stats["points_lbw_bowled"] = bowling_stats["lbw_bowled"] * 8
# Add the wicket bonus.
bowling_stats["points_wicket_bonus"] = bowling_stats["bonus_wickets"]
# Maiden overs give +12 points each.
bowling_stats["points_maiden"] = bowling_stats["maiden_overs"] * 12

# Total bowling points (before economy):
bowling_stats["bowling_points_no_econ"] = (
    bowling_stats["points_wickets"]
    + bowling_stats["points_lbw_bowled"]
    + bowling_stats["points_wicket_bonus"]
    + bowling_stats["points_maiden"]
)

# Calculate economy bonus.
econ_results = bowling_stats.apply(lambda row: economy_bonus(row["runs_conceded"], row["balls_bowled"]), axis=1)
bowling_stats["bonus_economy"], bowling_stats["economy_rate"] = zip(*econ_results)

# Total bowling points including economy bonus.
bowling_stats["bowling_points"] = bowling_stats["bowling_points_no_econ"] + bowling_stats["bonus_economy"]

print("\n=== Bowling Points Breakdown ===")
print(bowling_stats[["Bowler", "balls_bowled", "runs_conceded", "wickets", "lbw_bowled",
                     "bonus_wickets", "maiden_overs", "points_wickets", "points_lbw_bowled",
                     "points_wicket_bonus", "points_maiden", "bonus_economy", "economy_rate",
                     "bowling_points"]])



=== Bowling Points Breakdown ===
                Bowler  balls_bowled  runs_conceded  wickets  lbw_bowled  \
0      Abhishek Sharma            12             17      0.0         0.0   
1           Adam Zampa            24             47      1.0         0.0   
2     Fazalhaq Farooqi            18             49      0.0         0.0   
3        Harshal Patel            24             32      2.0         0.0   
4         Jofra Archer            24             73      0.0         0.0   
5   Maheesh Theekshana            24             47      2.0         0.0   
6       Mohammed Shami            18             32      1.0         0.0   
7          Nitish Rana             6              9      0.0         0.0   
8          Pat Cummins            24             57      0.0         0.0   
9       Sandeep Sharma            24             51      1.0         0.0   
10     Simarjeet Singh            18             46      2.0         0.0   
11    Tushar Deshpande            24             40   

In [8]:
# Fielding: Extract events from the entire dataset (df_sorted).

# Catches: where Dismissal Type is "caught"
catches_df = df_sorted[df_sorted["Dismissal Type"] == "caught"].groupby("Dismissal Fielder").size().reset_index(name="catches")

# Stumpings: where Dismissal Type is "stumped"
stumpings_df = df_sorted[df_sorted["Dismissal Type"] == "stumped"].groupby("Dismissal Fielder").size().reset_index(name="stumpings")

# Run outs: where Dismissal Type is "run out".
runouts_df = df_sorted[df_sorted["Dismissal Type"] == "run out"].copy()

# If there is a column to differentiate runout type (for example "Delivery Type" might not help here),
# then you could split based on that. In our case, we assume no extra column is available.
runouts_df = runouts_df.groupby("Dismissal Fielder").size().reset_index(name="runouts")

# Merge all fielding events.
fielding_stats = pd.merge(catches_df, stumpings_df, on="Dismissal Fielder", how="outer")
fielding_stats = pd.merge(fielding_stats, runouts_df, on="Dismissal Fielder", how="outer")
fielding_stats[["catches", "stumpings", "runouts"]] = fielding_stats[["catches", "stumpings", "runouts"]].fillna(0)

# Calculate fielding points:
fielding_stats["points_catches"] = fielding_stats["catches"] * 8
# Bonus: if catches >= 3, add a one-time bonus of +4 points.
fielding_stats["points_catch_bonus"] = fielding_stats["catches"].apply(lambda x: 4 if x >= 3 else 0)
fielding_stats["points_stumpings"] = fielding_stats["stumpings"] * 12
# For runouts we assign +6 per runout (adjust if you have a way to mark direct runouts).
fielding_stats["points_runouts"] = fielding_stats["runouts"] * 6

fielding_stats["fielding_points"] = (
    fielding_stats["points_catches"]
    + fielding_stats["points_catch_bonus"]
    + fielding_stats["points_stumpings"]
    + fielding_stats["points_runouts"]
)

print("\n=== Fielding Points Breakdown ===")
print(fielding_stats[["Dismissal Fielder", "catches", "stumpings", "runouts",
                      "points_catches", "points_catch_bonus", "points_stumpings",
                      "points_runouts", "fielding_points"]])



=== Fielding Points Breakdown ===
  Dismissal Fielder  catches  stumpings  runouts  points_catches  \
0   Abhinav Manohar        2        0.0      0.0              16   
1  Heinrich Klaasen        1        0.0      0.0               8   
2      Ishan Kishan        1        0.0      0.0               8   
3      Jofra Archer        1        0.0      0.0               8   
4       Pat Cummins        2        0.0      0.0              16   
5       Riyan Parag        2        0.0      0.0              16   
6   Shimron Hetmyer        1        0.0      0.0               8   
7  Yashasvi Jaiswal        2        0.0      0.0              16   

   points_catch_bonus  points_stumpings  points_runouts  fielding_points  
0                   0               0.0             0.0             16.0  
1                   0               0.0             0.0              8.0  
2                   0               0.0             0.0              8.0  
3                   0               0.0             

In [10]:
# --- Aggregating Batting Data per Player (across innings) ---
# Here, final_batting is grouped by [Innings, Striker] already.
# We now group by Striker to sum batting points across innings.
batting_agg = final_batting.groupby("Striker").agg(
    total_batting_points = pd.NamedAgg(column="batting_points", aggfunc="sum"),
    total_runs = pd.NamedAgg(column="final_score", aggfunc="sum"),
    total_balls_faced = pd.NamedAgg(column="balls_faced", aggfunc="sum"),
    total_boundaries = pd.NamedAgg(column="boundaries_total", aggfunc="sum"),
    total_sixes = pd.NamedAgg(column="sixes_total", aggfunc="sum"),
    avg_strike_rate = pd.NamedAgg(column="strike_rate", aggfunc="mean")
).reset_index()

# --- Assuming bowling_stats and fielding_stats are computed in earlier segments ---
# For example purposes, these dataframes include:
#   bowling_stats: with columns ["Bowler", "bowling_points", ...]
#   fielding_stats: with columns ["Dismissal Fielder", "fielding_points", ...]

# Build a master list of all players from batting, bowling, and fielding.
batting_players = set(batting_agg["Striker"])
bowling_players = set(bowling_stats["Bowler"])
fielding_players = set(fielding_stats["Dismissal Fielder"])
all_players = pd.DataFrame({"player": list(batting_players.union(bowling_players).union(fielding_players))})

# Merge batting aggregates.
all_players = pd.merge(all_players, batting_agg, left_on="player", right_on="Striker", how="left").drop("Striker", axis=1)

# Merge bowling stats.
all_players = pd.merge(all_players, bowling_stats[["Bowler", "bowling_points", "balls_bowled", "runs_conceded",
                                                    "wickets", "lbw_bowled", "maiden_overs", "bonus_economy", "economy_rate"]],
                        left_on="player", right_on="Bowler", how="left").drop("Bowler", axis=1)

# Merge fielding stats.
all_players = pd.merge(all_players, fielding_stats[["Dismissal Fielder", "fielding_points", "catches", "stumpings", "runouts"]],
                        left_on="player", right_on="Dismissal Fielder", how="left").drop("Dismissal Fielder", axis=1)

# Replace any missing values with 0.
cols_to_fill = ["total_batting_points", "total_runs", "total_balls_faced", "total_boundaries",
                "total_sixes", "avg_strike_rate", "bowling_points", "balls_bowled", "runs_conceded",
                "wickets", "lbw_bowled", "maiden_overs", "bonus_economy", "economy_rate",
                "fielding_points", "catches", "stumpings", "runouts"]

for col in cols_to_fill:
    if col in all_players.columns:
        all_players[col] = all_players[col].fillna(0)

# Compute the overall total fantasy points.
all_players["total_points"] = all_players["total_batting_points"] + all_players["bowling_points"] + all_players["fielding_points"]

print("\n=== Final Player Points Breakdown ===")
print(all_players[["player", "total_batting_points", "bowling_points", "fielding_points", "total_points"]])

# Optionally, save the final results.
all_players.to_csv("dream11_points_breakdown_final.csv", index=False)
print("\nFinal points have been saved to 'dream11_points_breakdown_final.csv'.")



=== Final Player Points Breakdown ===
                player  total_batting_points  bowling_points  fielding_points  \
0   Maheesh Theekshana                   0.0            46.0              0.0   
1          Travis Head                 100.0             0.0              0.0   
2       Mohammed Shami                   0.0            23.0              0.0   
3     Tushar Deshpande                   0.0            77.0              0.0   
4          Dhruv Jurel                 105.0             0.0              0.0   
5        Shubham Dubey                  53.0             0.0              0.0   
6     Fazalhaq Farooqi                   0.0            -6.0              0.0   
7       Sandeep Sharma                   0.0            19.0              0.0   
8           Adam Zampa                   0.0            21.0              0.0   
9      Simarjeet Singh                   0.0            44.0              0.0   
10        Ishan Kishan                 155.0             0.0          

In [43]:
import pandas as pd

# Load the CSV file.
x = pd.read_csv("dream11_points_breakdown_final.csv")
x.sort_values(by="total_points", ascending=False).head(11)
#Dream 11 team
#Captain: Ishan Kishan
#Vice Captain: Dhruv Jurel

,player,total_batting_points,total_runs,total_balls_faced,total_boundaries,total_sixes,avg_strike_rate,bowling_points,balls_bowled,runs_conceded,wickets,lbw_bowled,maiden_overs,bonus_economy,economy_rate,fielding_points,catches,stumpings,runouts,total_points
10,Ishan Kishan,155.0,106.0,47.0,11.0,6.0,225.531915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,8.0,1.0,0.0,0.0,163.0
4,Dhruv Jurel,105.0,70.0,35.0,5.0,6.0,200.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,105.0
1,Travis Head,100.0,67.0,31.0,9.0,3.0,216.129032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,100.0
12,Sanju Samson,99.0,66.0,37.0,7.0,4.0,178.378378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,99.0
3,Tushar Deshpande,0.0,0.0,0.0,0.0,0.0,0.000000,77.0,24.0,40.0,3.0,0.0,0.0,-2.0,10.00,0.0,0.0,0.0,0.0,77.0
13,Shimron Hetmyer,61.0,42.0,23.0,1.0,4.0,182.608696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,8.0,1.0,0.0,0.0,69.0
14,Heinrich Klaasen,51.0,34.0,14.0,5.0,1.0,242.857143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,8.0,1.0,0.0,0.0,59.0
5,Shubham Dubey,53.0,34.0,11.0,1.0,4.0,309.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,53.0
16,Harshal Patel,0.0,0.0,0.0,0.0,0.0,0.000000,50.0,24.0,32.0,2.0,0.0,0.0,0.0,8.00,0.0,0.0,0.0,0.0,50.0
11,Nitish Kumar Reddy,46.0,30.0,15.0,4.0,1.0,200.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,46.0


In [35]:
import pandas as pd

# Assume df is your DataFrame already loaded and cleaned.
# For example, you may have already run:
#   df = pd.read_csv("M2.csv")
#   df.columns = df.columns.str.strip()
# If you haven't lowercased the columns, continue with these exact names.

# ------------------------------------------------------------------
# 1. Filter for Legal Deliveries
# ------------------------------------------------------------------
# We define a legal delivery as one where both "Wides" and "No Balls" are zero.
legal_deliveries = df[(df["Wides"] == 0) & (df["No Balls"] == 0)].copy()

# ------------------------------------------------------------------
# 2. Group by Batter (Striker) and Bowler to Calculate Matchup Stats
# ------------------------------------------------------------------
# For each matchup, we calculate:
#   - The number of legal balls faced by the Striker (using "Over Ball")
#   - The total runs scored (using "Batter Runs")
# Then compute the strike rate as: (Total Runs / Balls Faced) * 100.
matchups = legal_deliveries.groupby(["Striker", "Bowler"]).agg(
    balls_played=("Over Ball", "count"),
    total_runs=("Batter Runs", "sum")
).reset_index()

matchups["strike_rate"] = (matchups["total_runs"] / matchups["balls_played"]) * 100

# ------------------------------------------------------------------
# 3. Filter and Identify Key Matchups
# ------------------------------------------------------------------
# To focus on meaningful battles, consider only matchups where the batter faced at least 6 legal deliveries.
matchups_filtered = matchups[matchups["balls_played"] >= 6]

# Identify dominant matchups (where the batter was very successful)
dominant_matchups = matchups_filtered.sort_values(by="strike_rate", ascending=False).head(3)

# Identify matchups where the batter struggled (low strike rate)
struggled_matchups = matchups_filtered.sort_values(by="strike_rate", ascending=True).head(3)

# ------------------------------------------------------------------
# 4. Output the Analysis
# ------------------------------------------------------------------
print("=== Top 3 Dominant Batter-vs-Bowler Matchups ===")
print(dominant_matchups)

print("\n=== Top 3 Batter-vs-Bowler Matchups Where the Batter Struggled ===")
print(struggled_matchups)


=== Top 3 Dominant Batter-vs-Bowler Matchups ===
         Striker          Bowler  balls_played  total_runs  strike_rate
43   Travis Head    Jofra Archer             6          22   366.666667
16  Ishan Kishan    Jofra Archer             9          27   300.000000
33  Sanju Samson  Mohammed Shami             6          18   300.000000

=== Top 3 Batter-vs-Bowler Matchups Where the Batter Struggled ===
            Striker         Bowler  balls_played  total_runs  strike_rate
6       Dhruv Jurel     Adam Zampa             7           3    42.857143
37  Shimron Hetmyer  Harshal Patel             8           9   112.500000
32     Sanju Samson  Harshal Patel             7           9   128.571429
